<p style="background: linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F); /* Gradient Background */
          font-family: 'Roboto', sans-serif;
          color: #F39C12; /* White Text for Contrast */
          font-size: 1.9rem;
          text-align: center;
          padding: 20px 35px;
          border-radius: 40px;
          box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
          letter-spacing: 1px;">
    SMS CLASSIFIER : SPAM OR HAM
</p>



<p align="center">
  <img src="img.jpg" alt="image" style="width: 500px;"/>
</p>

   <a id='top'></a>
<div class="list-group" id="list-tab" role="tablist">
<p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:150%;text-align:center;border-radius:20px 60px;">TABLE OF CONTENTS</p>   
    
* [1. Dataset](#1)
    
* [2. Importing Libraries](#2)
    
* [3. Loading Data](#3)
    
* [4. Visualization](#4)
     
* [5. Text Preprocessing](#5)
    
    * [5.1 Conversion to Lowercase](#5.1)
    * [5.2 Removing Urls](#5.2)
    * [5.3 Removing HTML Tags](#5.3)
    * [5.4 Removing Punctuation](#5.4)
    * [5.5 Replacing Shortconvo/Chatwords](#5.5)
    * [5.6 Spelling Correction](#5.6)
    * [5.7 Removing Stopwords](#5.7)
    * [5.8 Removing Emojis](#5.8)
    * [5.9 Lemmatization](#5.9)
    
* [6. Vectorization](#6) 
    * [6.1 Bag of Words](#6.1)
    * [6.2 TF-IDF](#6.2)
      
* [7. Data Preprocessing](#7)
    
* [8. Model Building](#8)
    
* [9. Model Evaluation](#9)


<a id="1"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">Dataset</p>   
https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset

<a id="2"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">Importing Libraries</p>   


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from nltk.corpus import stopwords
import string
from textblob import TextBlob
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

<a id="3"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">Loading Data</p>   


In [2]:
data=pd.read_csv("SMSSpamCollection", delimiter="\t",names=["label","message"])
data.head()

label                                            message
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...

<a id="4"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">Visualization</p>   


In [3]:
#Palette
cols= ["#F39C12", "#E598D8"] 
#first of all let us evaluate the target and find out if our data is imbalanced or not
plt.figure(figsize=(12,8))
fg = sns.countplot(x= data["label"], palette= cols)
fg.set_title("Count Plot of Classes", color="#58508d")
fg.set_xlabel("Classes", color="#58508d")
fg.set_ylabel("Number of Data points", color="#58508d")

C:\Users\SaadS\AppData\Local\Temp\ipykernel_20620\1615224628.py:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  fg = sns.countplot(x= data["label"], palette= cols)


Text(0, 0.5, 'Number of Data points')

<a id="5"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">Text Preprocessing</p>   


<a id="5.1"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">1. Conversion to Lowercase</p>   


In [4]:
data['message'] = data['message'].str.lower()

<a id="5.2"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">2. Removing Urls</p>   


In [5]:
def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)


In [6]:
data['message'] = data['message'].apply(remove_url)

<a id="5.3"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">3. Removing HTML Tags</p>   


In [7]:
def remove_html_tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text)



In [8]:
data['message'] = data['message'].apply(remove_html_tags)

<a id="5.4"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">4. Removing Punctuations</p>   


In [9]:
punct = string.punctuation
def remove_punct(text):
    return text.translate(str.maketrans('', '', punct))

In [10]:
data['message'] = data['message'].apply(remove_punct)

<a id="5.5"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">5. Replacing Short-Convo/Chat-Words</p>   


In [11]:
# Removing Short Convo / Chat Words
chat_words = {
    "AFAIK": "As Far As I Know",
    "AFK": "Away From Keyboard",
    "ASAP": "As Soon As Possible",
    "ATK": "At The Keyboard",
    "ATM": "At The Moment",
    "A3": "Anytime, Anywhere, Anyplace",
    "BAK": "Back At Keyboard",
    "BBL": "Be Back Later",
    "BBS": "Be Back Soon",
    "BFN": "Bye For Now",
    "B4N": "Bye For Now",
    "BRB": "Be Right Back",
    "BRT": "Be Right There",
    "BTW": "By The Way",
    "B4": "Before",
    "B4N": "Bye For Now",
    "CU": "See You",
    "CUL8R": "See You Later",
    "CYA": "See You",
    "FAQ": "Frequently Asked Questions",
    "FC": "Fingers Crossed",
    "FWIW": "For What It's Worth",
    "FYI": "For Your Information",
    "GAL": "Get A Life",
    "GG": "Good Game",
    "GN": "Good Night",
    "GMTA": "Great Minds Think Alike",
    "GR8": "Great!",
    "G9": "Genius",
    "IC": "I See",
    "ICQ": "I Seek you (also a chat program)",
    "ILU": "ILU: I Love You",
    "IMHO": "In My Honest/Humble Opinion",
    "IMO": "In My Opinion",
    "IOW": "In Other Words",
    "IRL": "In Real Life",
    "KISS": "Keep It Simple, Stupid",
    "LDR": "Long Distance Relationship",
    "LMAO": "Laugh My A.. Off",
    "LOL": "Laughing Out Loud",
    "LTNS": "Long Time No See",
    "L8R": "Later",
    "MTE": "My Thoughts Exactly",
    "M8": "Mate",
    "NRN": "No Reply Necessary",
    "OIC": "Oh I See",
    "PITA": "Pain In The A..",
    "PRT": "Party",
    "PRW": "Parents Are Watching",
    "QPSA?": "Que Pasa?",
    "ROFL": "Rolling On The Floor Laughing",
    "ROFLOL": "Rolling On The Floor Laughing Out Loud",
    "ROTFLMAO": "Rolling On The Floor Laughing My A.. Off",
    "SK8": "Skate",
    "STATS": "Your sex and age",
    "ASL": "Age, Sex, Location",
    "THX": "Thank You",
    "TTFN": "Ta-Ta For Now!",
    "TTYL": "Talk To You Later",
    "U": "You",
    "U2": "You Too",
    "U4E": "Yours For Ever",
    "WB": "Welcome Back",
    "WTF": "What The F...",
    "WTG": "Way To Go!",
    "WUF": "Where Are You From?",
    "W8": "Wait...",
    "7K": "Sick:-D Laugher",
    "TFW": "That feeling when",
    "MFW": "My face when",
    "MRW": "My reaction when",
    "IFYP": "I feel your pain",
    "TNTL": "Trying not to laugh",
    "JK": "Just kidding",
    "IDC": "I don't care",
    "ILY": "I love you",
    "IMU": "I miss you",
    "ADIH": "Another day in hell",
    "ZZZ": "Sleeping, bored, tired",
    "WYWH": "Wish you were here",
    "TIME": "Tears in my eyes",
    "BAE": "Before anyone else",
    "FIMH": "Forever in my heart",
    "BSAAW": "Big smile and a wink",
    "BWL": "Bursting with laughter",
    "BFF": "Best friends forever",
    "CSL": "Can't stop laughing"
}

In [12]:
def chat_conversion(text):
    new_text = []
    for i in text.split():
        if i.upper() in chat_words:
            new_text.append(chat_words[i.upper()])
        else:
            new_text.append(i)
    return " ".join(new_text)

In [13]:
data['message'] = data['message'].apply(chat_conversion)

<a id="5.6"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">6. Spelling Correction</p>   


In [14]:

def correct_spell(text):
    return str(TextBlob(text).correct())


In [15]:
data['message'] = data['message'].apply(correct_spell)

<a id="5.7"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">7. Removing Stopwords</p>   


In [16]:
stopword = set(stopwords.words('english'))
def remove_stopwords(text):
    new_text = []
    
    for word in text.split():
        if word in stopword:
            new_text.append('')
        else:
            new_text.append(word)
    x = new_text[:]
    new_text.clear()
    return " ".join(x)

In [17]:
data['message'] = data['message'].apply(remove_stopwords)

<a id="5.8"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">8. Removing Emojis</p>   


In [18]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [19]:
data['message'] = data['message'].apply(remove_emoji)

<a id="5.9"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">9. Lemmatization</p>   


In [20]:
lemmatizer = WordNetLemmatizer()

In [23]:
corpus = []
# just apply stemming i have already done text preprocessing
for i in range(0, len(data)):
    # Keep only alphabetic characters and split into words
    review = re.sub('[^a-zA-Z]', ' ', data['message'][i])
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if word not in stopword]
    review = ' '.join(review)
    corpus.append(review)

<a id="6"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">Vectorization</p>   


<a id="6.1"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">Bag of Words</p>   



In [24]:
cv = CountVectorizer(max_features=2500,ngram_range=(2,2))

In [25]:
X = cv.fit_transform(corpus).toarray()

In [26]:
cv.vocabulary_

{'got wat': 833,
 'ok war': 1472,
 'free entry': 608,
 'entry wily': 522,
 'wily come': 2326,
 'cup final': 328,
 'text receive': 2043,
 'txt rate': 2185,
 'rate apply': 1662,
 'you dun': 2416,
 'dont think': 479,
 'think go': 2100,
 'freeman hey': 628,
 'id like': 964,
 'like fun': 1158,
 'per request': 1516,
 'request selle': 1718,
 'selle selle': 1788,
 'selle minnaminunginte': 1787,
 'minnaminunginte nurungu': 1301,
 'nurungu vesta': 1442,
 'vesta set': 2225,
 'set callertune': 1831,
 'callertune caller': 191,
 'caller press': 190,
 'press copy': 1620,
 'copy friend': 311,
 'friend callertune': 631,
 'winner valued': 2340,
 'valued network': 2222,
 'network customer': 1401,
 'customer selected': 335,
 'selected received': 1786,
 'claim call': 239,
 'call claim': 151,
 'claim code': 240,
 'kl valid': 1032,
 'valid hour': 2219,
 'mobile month': 1329,
 'entitled update': 519,
 'update latest': 2200,
 'latest colour': 1105,
 'free call': 603,
 'call mobile': 170,
 'mobile update': 1335

<a id="6.2"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">TF-IDF</p>   


In [27]:
tfidf = TfidfVectorizer(max_features=2500,ngram_range=(1,2))
X_tfidf = tfidf.fit_transform(corpus).toarray()

In [28]:
tfidf.vocabulary_

{'go': 810,
 'point': 1597,
 'crazy': 445,
 'available': 106,
 'boris': 191,
 'great': 872,
 'world': 2388,
 'la': 1091,
 'line': 1162,
 'got': 856,
 'wat': 2312,
 'ok': 1470,
 'war': 2308,
 'joking': 1069,
 'you': 2428,
 'free': 745,
 'entry': 624,
 'wily': 2363,
 'come': 394,
 'win': 2364,
 'cup': 452,
 'final': 704,
 'must': 1398,
 'may': 1272,
 'text': 2071,
 'receive': 1704,
 'question': 1659,
 'txt': 2206,
 'rate': 1677,
 'apply': 70,
 'free entry': 750,
 'entry wily': 626,
 'txt rate': 2215,
 'rate apply': 1678,
 'dun': 581,
 'say': 1801,
 'early': 586,
 'already': 41,
 'you dun': 2442,
 'ah': 33,
 'dont': 545,
 'think': 2103,
 'life': 1152,
 'around': 82,
 'though': 2111,
 'dont think': 556,
 'freeman': 754,
 'hey': 934,
 'darling': 470,
 'week': 2330,
 'word': 2384,
 'back': 127,
 'id': 1004,
 'like': 1156,
 'fun': 769,
 'still': 1973,
 'xxx': 2404,
 'st': 1956,
 'send': 1836,
 'even': 637,
 'brother': 221,
 'speak': 1942,
 'treat': 2176,
 'per': 1532,
 'request': 1737,
 'sell

<a id="7"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">Data Preprocessing</p>   


### Using Bag of Words

In [29]:
# Independent Features
print(X)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [30]:
# Dependent Features or Output Features
y = pd.get_dummies(data['label']).astype(int)
y=y.iloc[:,0].values    
print(y)

[1 1 0 ... 1 1 1]


In [31]:
# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

### Using TF-IDF

In [32]:
# Independent Features
print(X_tfidf)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [33]:
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(X_tfidf, y, test_size=0.20, random_state=42)

<a id="8"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">Model Building</p>   


### Using Bag of Words

In [34]:
spam_detect_model = MultinomialNB().fit(X_train, y_train)

In [35]:
spam_detect_model

MultinomialNB()

In [36]:
y_pred=spam_detect_model.predict(X_test)

### Using TF-IDF

In [37]:
spam_detect_model_tfidf = MultinomialNB().fit(X_train_tfidf, y_train_tfidf)

In [38]:
spam_detect_model_tfidf

MultinomialNB()

In [39]:
y_pred_tfidf=spam_detect_model_tfidf.predict(X_test_tfidf)

<a id="9"></a>
# <p style="background:linear-gradient(135deg, #A2C2E4, #6C5B7B, #F1C40F);font-family:newtimeroman;color:#F39C12;font-size:100%;text-align:center;border-radius:20px 60px;">Model Evaluation</p>   


### Using Bag of Words

In [40]:
accuracy_score(y_test,y_pred)

0.9713004484304932

In [41]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.98      0.80      0.88       149
           1       0.97      1.00      0.98       966

    accuracy                           0.97      1115
   macro avg       0.98      0.90      0.93      1115
weighted avg       0.97      0.97      0.97      1115



### Using TF-IDF

In [42]:
accuracy_score(y_test_tfidf,y_pred_tfidf)

0.9757847533632287

In [43]:
print(classification_report(y_test_tfidf,y_pred_tfidf))

              precision    recall  f1-score   support

           0       0.98      0.83      0.90       149
           1       0.97      1.00      0.99       966

    accuracy                           0.98      1115
   macro avg       0.98      0.92      0.94      1115
weighted avg       0.98      0.98      0.97      1115

